In [25]:
import itertools
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA

# Charger les données
data = pd.read_csv('data.csv', parse_dates=True)
data['Date']=pd.to_datetime(data['Date'])
data.set_index('Date', inplace = True)
data.index = data.index.to_period('D')
data = data.dropna()

In [26]:
data.head()

,Bel Air,Bd. Rép.
Date,,
2010-01-01,39.45,51.21
2010-01-02,46.53,49.67
2010-01-03,37.03,44.08
2010-01-04,48.44,49.04
2010-01-05,39.79,40.58


In [18]:
data.tail()

,Bel Air,Bd. Rép.
Date,,
2013-11-28,19.05,11.96
2013-11-29,22.64,10.40
2013-11-30,21.74,19.09
2013-12-01,22.87,20.58
2013-12-02,30.90,6.83


In [19]:
import pandas as pd
from pmdarima.arima import auto_arima

# Utiliser auto_arima pour déterminer les ordres p, d et q avec le meilleur AIC
model = auto_arima(data['Bel Air'], 
                   start_p=0, max_p=8,
                   start_d=1, max_d=2,
                   start_q=0, max_q=8,
                   seasonal=False,
                   stepwise=True, trace=True)

# Afficher les ordres optimaux
print("Meilleurs ordres (AIC) : p={}, d={}, q={}".format(model.order[0], model.order[1], model.order[2]))


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=9645.927, Time=0.38 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=9647.124, Time=0.73 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=9646.417, Time=1.45 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=9643.927, Time=0.55 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=9487.580, Time=2.32 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=9452.974, Time=0.86 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=9595.483, Time=0.20 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=9454.794, Time=1.66 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=9454.851, Time=1.42 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=9455.436, Time=1.09 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=9553.420, Time=0.67 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=9456.949, Time=1.05 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=9451.100, Time=0.39 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=9485.738, Time=0.57 sec
 ARIMA(2,1,0)(0,0,0)[0]          

In [27]:
import pandas as pd
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Diviser les données en ensemble d'entraînement et ensemble de test
train_size = int(len(data) * 0.8)  # 80% des données pour l'entraînement
train, test = data[0:train_size], data[train_size:]

# Utiliser auto_arima pour déterminer les ordres p, d et q avec le meilleur AIC
model = auto_arima(train['Bel Air'], 
                   start_p=0, max_p=5,
                   start_d=1, max_d=2,
                   start_q=0, max_q=5,
                   seasonal=False,
                   stepwise=True, trace=True)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(n_periods=len(test))

# Calculer RMSE
rmse = sqrt(mean_squared_error(test['Bel Air'], predictions))

# Calculer R^2
r2 = r2_score(test['Bel Air'], predictions)

# Afficher les résultats
print("RMSE: {:.2f}".format(rmse))
print("R^2: {:.2f}".format(r2))
print("Ordres optimaux (AIC): p={}, d={}, q={}".format(model.order[0], model.order[1], model.order[2]))


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=9017.844, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=7784.276, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=8356.294, Time=0.07 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=7785.909, Time=0.11 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=7785.680, Time=0.12 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=7784.208, Time=0.18 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=7680.236, Time=0.46 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=7762.042, Time=0.17 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=7682.156, Time=0.50 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=7682.176, Time=0.69 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=7681.942, Time=0.62 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=7741.979, Time=0.18 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=7682.122, Time=1.00 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=7674.968, Time=1.08 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept

In [29]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima



# Diviser les données en ensemble d'entraînement et ensemble de test
train_size = int(len(data) * 0.8)  # 80% des données pour l'entraînement
train, test = data[0:train_size], data[train_size:]

# Utiliser auto_arima pour déterminer les ordres p, d et q avec le meilleur AIC
model = auto_arima(train['Bel Air'], 
                   start_p=0, max_p=5,
                   start_d=1, max_d=2,
                   start_q=0, max_q=5,
                   seasonal=False,
                   stepwise=True, trace=True)

# Faire des prédictions sur l'ensemble de test
forecast_steps = len(test)
predictions, conf_int = model.predict(n_periods=forecast_steps, return_conf_int=True)

# Créer un index de dates pour les prédictions
prediction_index = pd.date_range(start=test.index[0], periods=forecast_steps, freq='D')

# Assigner l'index aux prédictions
predictions = pd.Series(predictions, index=prediction_index)
conf_int = pd.DataFrame(conf_int, index=prediction_index, columns=['lower', 'upper'])

# Calculer le coefficient de corrélation
correlation = np.corrcoef(test['Bel Air'], predictions)[0, 1]

# Afficher le coefficient de corrélation et les ordres optimaux (AIC)
print("Coefficient de corrélation : {:.2f}".format(correlation))
print("Ordres optimaux (AIC): p={}, d={}, q={}".format(model.order[0], model.order[1], model.order[2]))


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=9017.844, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=7784.276, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=8356.294, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=7785.909, Time=0.06 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=7785.680, Time=0.10 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=7784.208, Time=0.11 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=7680.236, Time=0.40 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=7762.042, Time=0.14 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=7682.156, Time=0.52 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=7682.176, Time=0.62 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=7681.942, Time=0.60 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=7741.979, Time=0.17 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=7682.122, Time=0.91 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=7674.968, Time=1.06 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept

ValueError: Cannot convert Period to Timestamp unambiguously. Use to_timestamp

In [23]:
import pandas as pd
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Charger les données

# Diviser les données en ensemble d'entraînement et ensemble de test
train_size = int(len(data) * 0.8)  # 80% des données pour l'entraînement
train, test = data[0:train_size], data[train_size:]

# Utiliser auto_arima pour déterminer les ordres p, d, et q avec le meilleur AIC
model = auto_arima(train['Bel Air'], 
                   start_p=0, max_p=5,
                   start_d=1, max_d=2,
                   start_q=0, max_q=5,
                   seasonal=False,
                   stepwise=True, trace=True)

# Faire des prédictions sur l'ensemble de test
forecast_steps = len(test)
predictions = model.predict(n_periods=forecast_steps)

# Assigner l'index des prédictions
predictions_index = test.index[:forecast_steps]

# Calculer RMSE
rmse = sqrt(mean_squared_error(test['Bel Air'][:forecast_steps], predictions))

# Calculer R^2
r2 = r2_score(test['Bel Air'][:forecast_steps], predictions)

# Afficher les résultats
print("RMSE: {:.2f}".format(rmse))
print("R^2: {:.2f}".format(r2))
print("Ordres optimaux (AIC): p={}, d={}, q={}".format(model.order[0], model.order[1], model.order[2]))


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=9017.844, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=7784.276, Time=0.10 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=8356.294, Time=0.14 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=7785.909, Time=0.15 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=7785.680, Time=0.28 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=7784.208, Time=0.29 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=7680.236, Time=0.89 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=7762.042, Time=0.37 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=7682.156, Time=1.11 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=7682.176, Time=1.45 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=7681.942, Time=1.39 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=7741.979, Time=0.38 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=7682.122, Time=2.04 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=7674.968, Time=2.38 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept

C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [30]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import itertools



# Définir une plage de valeurs possibles pour p, d et q
p_range = range(0, 4)  # Valeurs possibles pour p
d_range = range(0, 2)  # Valeurs possibles pour d
q_range = range(0, 4)  # Valeurs possibles pour q

# Générer toutes les combinaisons possibles de p, d et q
pdq_combinations = list(itertools.product(p_range, d_range, q_range))

best_aic = float("inf")  # Initialiser l'AIC à l'infini
best_bic = float("inf")  # Initialiser le BIC à l'infini
best_order_aic = None
best_order_bic = None

# Parcourir toutes les combinaisons possibles de p, d et q
for order in pdq_combinations:
    try:
        # Créer et ajuster le modèle ARIMA avec l'ordre actuel
        model = ARIMA(data['Bel Air'], order=order)
        results = model.fit()

        # Calculer l'AIC et le BIC pour le modèle actuel
        aic = results.aic
        bic = results.bic

        # Mettre à jour les ordres optimaux si l'AIC ou le BIC actuel est meilleur que le précédent meilleur
        if aic < best_aic:
            best_aic = aic
            best_order_aic = order

        if bic < best_bic:
            best_bic = bic
            best_order_bic = order

    except:
        continue

# Afficher les ordres optimaux selon l'AIC et le BIC
print("Meilleur ordre ARIMA basé sur l'AIC:", best_order_aic, "AIC:", best_aic)
print("Meilleur ordre ARIMA basé sur le BIC:", best_order_bic, "BIC:", best_bic)


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to conver

Meilleur ordre ARIMA basé sur l'AIC: (2, 1, 1) AIC: 9451.100473721272
Meilleur ordre ARIMA basé sur le BIC: (2, 1, 1) BIC: 9470.872963412548


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [31]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import itertools

# Charger les données (remplacez ceci par votre propre ensemble de données)
# data = pd.read_csv('votre_fichier.csv')

# Définir une plage de valeurs possibles pour p, d et q
p_range = range(0, 5)  # Valeurs possibles pour p
d_range = range(0, 2)  # Valeurs possibles pour d
q_range = range(0, 5)  # Valeurs possibles pour q

# Générer toutes les combinaisons possibles de p, d et q
pdq_combinations = list(itertools.product(p_range, d_range, q_range))

best_aic = float("inf")  # Initialiser l'AIC à l'infini
best_bic = float("inf")  # Initialiser le BIC à l'infini
best_order_aic = None
best_order_bic = None

# Afficher toutes les combinaisons possibles
print("Toutes les combinaisons possibles de p, d, et q :")
for order in pdq_combinations:
    print("p =", order[0], ", d =", order[1], ", q =", order[2])

    try:
        # Créer et ajuster le modèle ARIMA avec l'ordre actuel
        model = ARIMA(data['Bel Air'], order=order)
        results = model.fit()

        # Calculer l'AIC et le BIC pour le modèle actuel
        aic = results.aic
        bic = results.bic

        # Mettre à jour les ordres optimaux si l'AIC ou le BIC actuel est meilleur que le précédent meilleur
        if aic < best_aic:
            best_aic = aic
            best_order_aic = order

        if bic < best_bic:
            best_bic = bic
            best_order_bic = order

    except Exception as e:
        print("Erreur rencontrée:", e)
        continue

# Afficher les ordres optimaux selon l'AIC et le BIC
print("\nMeilleur ordre ARIMA basé sur l'AIC:", best_order_aic, "AIC:", best_aic)
print("Meilleur ordre ARIMA basé sur le BIC:", best_order_bic, "BIC:", best_bic)


Toutes les combinaisons possibles de p, d, et q :
p = 0 , d = 0 , q = 0
p = 0 , d = 0 , q = 1
p = 0 , d = 0 , q = 2
p = 0 , d = 0 , q = 3
p = 0 , d = 0 , q = 4
p = 0 , d = 1 , q = 0
p = 0 , d = 1 , q = 1
p = 0 , d = 1 , q = 2
p = 0 , d = 1 , q = 3
p = 0 , d = 1 , q = 4
p = 1 , d = 0 , q = 0
p = 1 , d = 0 , q = 1
p = 1 , d = 0 , q = 2
p = 1 , d = 0 , q = 3
p = 1 , d = 0 , q = 4
p = 1 , d = 1 , q = 0
p = 1 , d = 1 , q = 1


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


p = 1 , d = 1 , q = 2
p = 1 , d = 1 , q = 3
p = 1 , d = 1 , q = 4
p = 2 , d = 0 , q = 0
p = 2 , d = 0 , q = 1
p = 2 , d = 0 , q = 2


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


p = 2 , d = 0 , q = 3
p = 2 , d = 0 , q = 4
p = 2 , d = 1 , q = 0
p = 2 , d = 1 , q = 1
p = 2 , d = 1 , q = 2
p = 2 , d = 1 , q = 3
p = 2 , d = 1 , q = 4
p = 3 , d = 0 , q = 0
p = 3 , d = 0 , q = 1
p = 3 , d = 0 , q = 2
p = 3 , d = 0 , q = 3


...

C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


p = 4 , d = 1 , q = 0
p = 4 , d = 1 , q = 1
p = 4 , d = 1 , q = 2
p = 4 , d = 1 , q = 3


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


p = 4 , d = 1 , q = 4

Meilleur ordre ARIMA basé sur l'AIC: (2, 1, 1) AIC: 9451.100473721272
Meilleur ordre ARIMA basé sur le BIC: (2, 1, 1) BIC: 9470.872963412548


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [33]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import itertools

# Définir une plage de valeurs possibles pour p, d et q
p_range = range(0, 6)  # Valeurs possibles pour p
d_range = range(0, 2)  # Valeurs possibles pour d
q_range = range(0, 6)  # Valeurs possibles pour q

# Générer toutes les combinaisons possibles de p, d et q
pdq_combinations = list(itertools.product(p_range, d_range, q_range))

# Initialiser une liste pour stocker les résultats AIC et BIC
results_list = []

# Parcourir toutes les combinaisons possibles de p, d et q
for order in pdq_combinations:
    try:
        # Créer et ajuster le modèle ARIMA avec l'ordre actuel
        model = ARIMA(data['Bel Air'], order=order)
        results = model.fit()

        # Calculer l'AIC et le BIC pour le modèle actuel
        aic = results.aic
        bic = results.bic
        
        # Ajouter les résultats à la liste
        results_list.append({'p': order[0], 'd': order[1], 'q': order[2], 'AIC': aic, 'BIC': bic})

    except:
        continue

# Créer un DataFrame à partir de la liste de dictionnaires
results_df = pd.DataFrame(results_list)

# Afficher le DataFrame
print(results_df)


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to conver

    p  d  q           AIC           BIC
0   0  0  0  10314.043211  10323.931385
1   0  0  1   9732.286175   9747.118436
2   0  0  2   9566.340398   9586.116747
3   0  0  3   9527.124200   9551.844636
4   0  0  4   9519.293807   9548.958331
.. .. .. ..           ...           ...
67  5  1  1   9454.971163   9489.573020
68  5  1  2   9456.913459   9496.458439
69  5  1  3   9458.712418   9503.200520
70  5  1  4   9460.294221   9509.725445
71  5  1  5   9462.363564   9516.737910

[72 rows x 5 columns]


In [34]:
results_df

,p,d,q,AIC,BIC
0,0,0,0,10314.043211,10323.931385
1,0,0,1,9732.286175,9747.118436
2,0,0,2,9566.340398,9586.116747
3,0,0,3,9527.124200,9551.844636
4,0,0,4,9519.293807,9548.958331
...,...,...,...,...,...
67,5,1,1,9454.971163,9489.573020
68,5,1,2,9456.913459,9496.458439
69,5,1,3,9458.712418,9503.200520
70,5,1,4,9460.294221,9509.725445


In [35]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import itertools

# Définir une plage de valeurs possibles pour p, d et q
p_range = range(0, 4)  # Valeurs possibles pour p
d_range = range(0, 2)  # Valeurs possibles pour d
q_range = range(0, 4)  # Valeurs possibles pour q

# Générer toutes les combinaisons possibles de p, d et q
pdq_combinations = list(itertools.product(p_range, d_range, q_range))

# Initialiser une liste pour stocker les résultats
results_list = []

# Parcourir toutes les combinaisons possibles de p, d et q
for order in pdq_combinations:
    try:
        # Créer et ajuster le modèle ARIMA avec l'ordre actuel
        model = ARIMA(data['Bel Air'], order=order)
        results = model.fit()

        # Calculer l'AIC et le BIC pour le modèle actuel
        aic = results.aic
        bic = results.bic
        
        # Ajouter les résultats à la liste
        results_list.append({'p': order[0], 'd': order[1], 'q': order[2], 'AIC': aic, 'BIC': bic})

        # Afficher les résultats pour chaque combinaison
        print("Combinaison: p =", order[0], ", d =", order[1], ", q =", order[2], ", AIC =", aic, ", BIC =", bic)

    except:
        continue

# Créer un DataFrame à partir de la liste de dictionnaires
results_df = pd.DataFrame(results_list)

# Afficher le DataFrame
print(results_df)

# Trouver et afficher la meilleure combinaison basée sur l'AIC
best_aic_combination = results_df.loc[results_df['AIC'].idxmin()]
print("Meilleure combinaison basée sur l'AIC:", best_aic_combination)

# Trouver et afficher la meilleure combinaison basée sur le BIC
best_bic_combination = results_df.loc[results_df['BIC'].idxmin()]
print("Meilleure combinaison basée sur le BIC:", best_bic_combination)


Combinaison: p = 0 , d = 0 , q = 0 , AIC = 10314.043210667094 , BIC = 10323.931385083553
Combinaison: p = 0 , d = 0 , q = 1 , AIC = 9732.286174668134 , BIC = 9747.118436292822
Combinaison: p = 0 , d = 0 , q = 2 , AIC = 9566.340397909524 , BIC = 9586.116746742442
Combinaison: p = 0 , d = 0 , q = 3 , AIC = 9527.124199563325 , BIC = 9551.844635604471
Combinaison: p = 0 , d = 0 , q = 4 , AIC = 9519.293807492468 , BIC = 9548.958330741845
Combinaison: p = 0 , d = 1 , q = 0 , AIC = 9643.926861909327 , BIC = 9648.869984332146
Combinaison: p = 0 , d = 1 , q = 1 , AIC = 9644.4170700225 , BIC = 9654.303314868139
Combinaison: p = 0 , d = 1 , q = 2 , AIC = 9529.23919747807 , BIC = 9544.068564746527
Combinaison: p = 0 , d = 1 , q = 3 , AIC = 9458.666719548819 , BIC = 9478.439209240096
Combinaison: p = 0 , d = 1 , q = 4 , AIC = 9452.00119185137 , BIC = 9476.716803965468
Combinaison: p = 1 , d = 0 , q = 0 , AIC = 9509.858037792292 , BIC = 9524.69029941698
Combinaison: p = 1 , d = 0 , q = 1 , AIC = 949

C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Combinaison: p = 1 , d = 1 , q = 1 , AIC = 9485.737731006968 , BIC = 9500.567098275425
Combinaison: p = 1 , d = 1 , q = 2 , AIC = 9453.573676583415 , BIC = 9473.346166274692
Combinaison: p = 1 , d = 1 , q = 3 , AIC = 9451.961269391115 , BIC = 9476.676881505213
Combinaison: p = 1 , d = 1 , q = 4 , AIC = 9453.877750462838 , BIC = 9483.536484999755
Combinaison: p = 2 , d = 0 , q = 0 , AIC = 9496.731018350587 , BIC = 9516.507367183505
Combinaison: p = 2 , d = 0 , q = 1 , AIC = 9495.161220035428 , BIC = 9519.881656076575


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Combinaison: p = 2 , d = 0 , q = 2 , AIC = 9460.291666283882 , BIC = 9489.956189533259
Combinaison: p = 2 , d = 0 , q = 3 , AIC = 9457.800322714016 , BIC = 9492.408933171622
Combinaison: p = 2 , d = 0 , q = 4 , AIC = 9459.589068134548 , BIC = 9499.141765800385
Combinaison: p = 2 , d = 1 , q = 0 , AIC = 9593.483483839558 , BIC = 9608.312851108016
Combinaison: p = 2 , d = 1 , q = 1 , AIC = 9451.100473721272 , BIC = 9470.872963412548
Combinaison: p = 2 , d = 1 , q = 2 , AIC = 9452.979107004781 , BIC = 9477.69471911888
Combinaison: p = 2 , d = 1 , q = 3 , AIC = 9453.947348144424 , BIC = 9483.606082681341
Combinaison: p = 2 , d = 1 , q = 4 , AIC = 9455.776014629286 , BIC = 9490.377871589022
Combinaison: p = 3 , d = 0 , q = 0 , AIC = 9489.529577222454 , BIC = 9514.250013263601
Combinaison: p = 3 , d = 0 , q = 1 , AIC = 9456.834992880888 , BIC = 9486.499516130265
Combinaison: p = 3 , d = 0 , q = 2 , AIC = 9458.76513371434 , BIC = 9493.373744171946


...

C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Combinaison: p = 4 , d = 0 , q = 4 , AIC = 9461.534979693908 , BIC = 9510.975851776204
Combinaison: p = 4 , d = 1 , q = 0 , AIC = 9533.343246366538 , BIC = 9558.058858480636
Combinaison: p = 4 , d = 1 , q = 1 , AIC = 9453.053991283121 , BIC = 9482.712725820038
Combinaison: p = 4 , d = 1 , q = 2 , AIC = 9453.938097460663 , BIC = 9488.5399544204


C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Combinaison: p = 4 , d = 1 , q = 3 , AIC = 9456.334592244608 , BIC = 9495.879571627163
Combinaison: p = 4 , d = 1 , q = 4 , AIC = 9456.09991376979 , BIC = 9500.588015575166
    p  d  q           AIC           BIC
0   0  0  0  10314.043211  10323.931385
1   0  0  1   9732.286175   9747.118436
2   0  0  2   9566.340398   9586.116747
3   0  0  3   9527.124200   9551.844636
4   0  0  4   9519.293807   9548.958331
5   0  1  0   9643.926862   9648.869984
6   0  1  1   9644.417070   9654.303315
7   0  1  2   9529.239197   9544.068565
8   0  1  3   9458.666720   9478.439209
9   0  1  4   9452.001192   9476.716804
10  1  0  0   9509.858038   9524.690299
11  1  0  1   9493.670572   9513.446921
12  1  0  2   9493.116348   9517.836784
13  1  0  3   9463.010439   9492.674962
14  1  0  4   9457.617687   9492.226298
15  1  1  0   9645.124109   9655.010354
16  1  1  1   9485.737731   9500.567098
17  1  1  2   9453.573677   9473.346166
18  1  1  3   9451.961269   9476.676882
19  1  1  4   9453.877750  

C:\Users\NZO BUSINESS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
